# Import modules

In [ ]:
import stlab
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy
from scipy.optimize import *
import time
import sys 
import io
import os
from IPython.display import display, Javascript
from shutil import *

from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
from Configuration_KNO1 import config, RR_1_IF,RO_lo, readout_len

# Description

Script for doing ringdown spectrosocpy of a linear resonator. A measurement pulse of a certain length is sent to the resonator. The response of the resonator is measured during and after the pulse allowing the observation of energy relaxation. The time-of-flight of the readout pulse is increased. This ensures that data will also be acquired after the application of the readout pulse. 

**Note: at this point the ringdown_analysis() function might not be as general and optimal as it could be. This might lead to unexpected results. Future improvements should/could include:** 
* Changing filter: use blocks instead of Gaussians or make use of scipy to design filters 

# Configure QM unit

In [ ]:
qmm = QuantumMachinesManager()
qm = qmm.open_qm(config)

# QUA Program

In [ ]:
a = 1.75
n_max = 5000

with program() as ringdown:
    
    ind = declare(int)
    n = declare(int)

    adc_stream = declare_stream(adc_trace=True)
    
    with for_(n, 0, n<n_max, n+1):
        reset_phase("RR_1")
        wait(int(2000/4), "RR_1")
        measure("readout"*amp(a), "RR_1", adc_stream) 
        
    #Save raw ADC stream
    with stream_processing():
        adc_stream.input1().average().save("adc1")
        adc_stream.input2().average().save("adc2")    

In [ ]:
job = qm.execute(ringdown, duration_limit=0, data_limit=0) 

In [ ]:
res_handles= job.result_handles
res_handles.wait_for_all_values()

# Fetch data and save to .dat

In [ ]:
#fetch data
adc1_handle = res_handles.get('adc1')
adc2_handle = res_handles.get('adc2')

adc1_data=adc1_handle.fetch_all() 
adc2_data=adc2_handle.fetch_all()

In [ ]:
prefix = 'S' #prefix for measurement folder name.  Can be anything or empty
idstring = f'R1_Ringdown_Spec_RRlo={RO_lo*10**(-9)}GHz_RRIF={RR_1_IF*10**(-6)}MHz_a={a}_n={n_max}'

data=np.asarray([adc1_data.T,adc2_data.T])

data_dict={'adc1':data[0],
           'adc2':data[1],
    }


old_stdout = sys.stdout
new_stdout = io.StringIO()
sys.stdout = new_stdout

myfile=stlab.newfile(prefix,idstring,data_dict.keys(),autoindex=True)
output = new_stdout.getvalue()
sys.stdout = old_stdout
print(output)
M_ind = output.find("Measurement Name")
M_name = output[M_ind+len('Measurement Name:  '):-1]

stlab.savedict(myfile,data_dict)

## Processing of Raw ADC

In [ ]:
def Gauss_filt(f,sigma,d1,d2):
    '''
    Very basic frequency domain filters consisting of two Gaussians centered at d1 and d2, respectively, with 
    standard deviation sigma.
    '''
    return np.exp(-((f-d1)/sigma)**2) + np.exp(-((f-d2)/sigma)**2)

def Ringdown_analysis(adc1_dat,adc2_dat, readout_len, RR_1_IF,sigma, delta,a):
    '''
    Function that performs filtering, demodulation and fitting of raw ADC signals that measure the ringdown of a cavity.
    '''
    adc1 = adc1_data/2**12
    adc2 = adc1_data/2**12
    plt.figure(num=None, figsize=(10, 8), dpi=120,constrained_layout=True)
    plt.plot(adc1)
    f_IF_r = RR_1_IF*10**(-6)

    #Fourier transform
    adc1_fft = np.fft.fft(adc1)
    adc2_fft = np.fft.fft(adc2)

    #Bandpass around intermediate frequency
    f_arr1 = np.linspace(0,readout_len,readout_len)
    
    #Note: need to automatically find frequency where adc signals peak, this is not exaclty at the IF frequency
    adc1_f_ft = adc1_fft*Gauss_filt(f_arr1,sigma,f_IF_r,readout_len-f_IF_r)     
    adc2_f_ft = adc2_fft*Gauss_filt(f_arr1,sigma,f_IF_r,readout_len-f_IF_r)
    
    #f_vec = np.fft.fftfreq(readout_len, 1e-9)*10**(-6)
    #f_vec = np.linspace(0,1000,1000)
    #plt.plot(f_arr1,np.abs(adc1_f_ft))
    #plt.plot(f_arr1,Gauss_filt(f_arr1,sigma,f_IF_r,readout_len-f_IF_r))
    #plt.xlim(120,180)
    
    #Filtered adc signals
    adc1_f = np.real(np.fft.ifft(adc1_f_ft))
    adc2_f = np.real(np.fft.ifft(adc2_f_ft))
    plt.plot(adc1_f)
    
    #Specify demodulation frequency and weights
    t_cut = readout_len - readout_len//10
    t = np.linspace(0,t_cut,t_cut)
    w_demod = (f_IF_r+delta)*(2*np.pi)*10**(-3)

    Integ_cos = np.cos(w_demod*t)
    Integ_sin = -np.sin(w_demod*t)
    
    #Demodulate filtered adc signals
    A = Integ_cos*adc1_f[0:t_cut]
    B = Integ_sin*adc2_f[0:t_cut]
    iA = Integ_sin*adc1_f[0:t_cut]
    iB = Integ_cos*adc2_f[0:t_cut]

    I_uf = A-B 
    Q_uf = iA+iB
    
    #Demodulate and apply lPF to I and Q
    f_arr2 = np.linspace(0,t_cut,t_cut)
    Q_fft = np.fft.fft(Q_uf)
    Q_fft_f = Q_fft*Gauss_filt(f_arr2,sigma,delta,t_cut)
    Q_f = np.real(np.fft.ifft(Q_fft_f))[t_cut//20:t_cut-t_cut//10]

    I_fft = np.fft.fft(I_uf)
    I_fft_f = I_fft*Gauss_filt(f_arr2,sigma,delta,t_cut)
    I_f = np.real(np.fft.ifft(I_fft_f))[t_cut//20:t_cut-t_cut//10]
    
    plt.figure(num=None, figsize=(10, 8), dpi=120,constrained_layout=True)
    plt.plot(I_f)
    
    
    #Fit and plot
    t_plot = np.arange(t_cut//20,t_cut-t_cut//10,1)
    s_dat = np.abs(I_f+1j*Q_f)

    def IQ_fit(t,kappa,A,B): 
        return A*np.exp(-kappa/2*t)+B       
    
    popt, pcov = curve_fit(IQ_fit, t_plot, s_dat, p0 = [1/200,0.001,0])
    kappa = popt[0]*10**3/(2*np.pi)
    print('kappa =', kappa, 'MHz')
                              
    plt.figure(num=None, figsize=(10, 8), dpi=120,constrained_layout=True)
    plt.tight_layout()
    plt.plot(t_plot,I_f,label = 'I')
    plt.plot(t_plot,Q_f,label ='Q')
    plt.scatter(t_plot[0:-1:20],s_dat[0:-1:20],label ='|I+iQ|', s=15, c = 'green')
    plt.plot(t_plot,IQ_fit(t_plot,popt[0],popt[1],popt[2]),label = 'fit', ls='--')
    plt.xlim(t_plot[0],t_plot[-1])
    plt.legend(fontsize = 16, loc = 'best')
    plt.axhline(0,ls='--',alpha = 0.8, c ='black')
    plt.xlabel('t (ns)',fontsize = 16)
    plt.ylabel('Output (a.u.)',fontsize = 16)
    plt.xticks(fontsize = 16);plt.yticks(fontsize = 16)
    plt.title(r'Ringdown, amp = %.1f, $\Delta$ = %.1f MHz, $\sigma$ = %.0f, $\kappa$ = %.4f MHz' % (a, delta, sigma,kappa), fontsize =16)
    
    return kappa

In [ ]:
#Create directory in measurement folder to save plots
meas_path = os.path.join(os.getcwd(),M_name)
plot_path = os.path.join(meas_path,'plots')
os.mkdir(plot_path)

In [ ]:
adc1_dat = adc1_data
adc2_dat = adc2_data
sigma = 10              #Note: values of kappa depend on the value of sigma. Its probably better to implement a different kind
                        #of filter, e.g. rounded blocks instead of Gaussians
delta = 0

kappa = Ringdown_analysis(adc1_dat,adc2_dat, readout_len, RR_1_IF,sigma, delta,a)

#plt.savefig(plot_path+f'/Ringdown_amp={a}_delta={delta}MHz_sigma={sigma}_kappa={round(kappa,4)}MHz.png')

# Save this file and configuration file to measurement folder

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
#define document paths
meas_path = os.path.join(os.getcwd(),M_name)

current_nb_path = os.path.join(os.getcwd(),nb_name)
save_nb_path = os.path.join(meas_path,nb_name)

current_config_path = os.path.join(os.getcwd(), 'Configuration_KNO1.py')
save_config_path = os.path.join(meas_path, 'Configuration_KNO1.py')

#save notebook
display(Javascript('IPython.notebook.save_checkpoint();'))

#copy to measurement folder 
copy2(current_nb_path,save_nb_path);
copy2(current_config_path,save_config_path);